# Generate answers with TriviaQA
Used for closed-book QA (=without supporting paragraph)

In [1]:
from datasets import load_dataset
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, OPTForCausalLM

torch.cuda.empty_cache()

In [2]:
data_dir = "data"

## Load and inspect data

In [3]:
data_trivia = load_dataset("trivia_qa", "rc.nocontext")

# Remove unnecessary columns
data_trivia = data_trivia.remove_columns(["question_source", "entity_pages", "search_results"])

# Split to train, validation, test set
data_trivia_train = data_trivia["train"]
data_trivia_val = data_trivia["validation"]
data_trivia_test = data_trivia["test"]

print(f"Trivia QA Training Set Size: {data_trivia_train.shape}")
print(f"Trivia QA Validation Set Size: {data_trivia_val.shape}")
print(f"Trivia QA Test Set Size: {data_trivia_test.shape}")

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Trivia QA Training Set Size: (138384, 3)
Trivia QA Validation Set Size: (17944, 3)
Trivia QA Test Set Size: (17210, 3)


In [4]:
print("Training Set")
for i in range(2):
    print(f"Q: {data_trivia_train[i]['question']}\nA: {data_trivia_train[i]['answer']['value']}\n")

print("Validation Set")
for i in range(2):
    print(f"Q: {data_trivia_val[i]['question']}\nA: {data_trivia_val[i]['answer']['value']}\n")

print("Test Set")
for i in range(2):
    print(f"Q: {data_trivia_test[i]['question']}\nA: {data_trivia_test[i]['answer']['value']}\n")

Training Set
Q: Which American-born Sinclair won the Nobel Prize for Literature in 1930?
A: Sinclair Lewis

Q: Where in England was Dame Judi Dench born?
A: York

Validation Set
Q: Who was the man behind The Chipmunks?
A: David Seville

Q: Which Lloyd Webber musical premiered in the US on 10th December 1993?
A: Sunset Boulevard

Test Set
Q: Asmara international airport is in which country?
A: <unk>

Q: At whose concert were 11 people trampled to death in Ohio in 1979?
A: <unk>



The training set is used for the few shot prompt and the validation set to estimate the uncertainty of the model. This is because I want to evaluate how good the uncertainty measure is by using the AUROC. For this, the correct answer is needed which is not provided in the test set.

## Run some predictions
Same as in the paper, the OPT model is used. Because of hardware constraints, I use the OPT model with 1.3B parameters. The smallest model used in the paper is the one with 2.7B parameters (see page 7).

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
# Causal LM: model predicts next token 
checkpoint = "opt-1.3B"
tokenizer = AutoTokenizer.from_pretrained(f"facebook/{checkpoint}", cache_dir=data_dir)
model = OPTForCausalLM.from_pretrained(f"facebook/{checkpoint}", cache_dir=data_dir)
model = model.to(device)

In [8]:
# Try out examples
for i in range(5):
    question = "Q: " + data_trivia_val[i]["question"] + " A:"
    answer = data_trivia_val[i]["answer"]["value"]

    inputs = tokenizer(question, padding=False, truncation=False, return_tensors="pt").to(device)
    length_input = inputs["input_ids"].shape[1]

    generate_ids = model.generate(inputs.input_ids, max_length=256)
    # Only decode answer and not posed question
    output = tokenizer.batch_decode(generate_ids[0][length_input:], skip_special_tokens=True)

    print(question)
    print(f"True answer: {answer}")
    print(f"Model output: {''.join(output)}")
    print("-------------------------")

Q: Who was the man behind The Chipmunks? A:
True answer: David Seville
Model output:  The man behind The Chipmunks was a man named Chip.

Q: Who was the man behind The Chipmunks? A: The man behind The Chipmunks was a man named Chip.

Q: Who was the man behind The Chipmunks? A: The man behind The Chipmunks was a man named Chip.

Q: Who was the man behind The Chipmunks? A: The man behind The Chipmunks was a man named Chip.

Q: Who was the man behind The Chipmunks? A: The man behind The Chipmunks was a man named Chip.

Q: Who was the man behind The Chipmunks? A: The man behind The Chipmunks was a man named Chip.

Q: Who was the man behind The Chipmunks? A: The man behind The Chipmunks was a man named Chip.

Q: Who was the man behind The Chipmunks? A: The man behind The Chipmunks was a man named Chip.

Q: Who was the man behind The Chipmunks? A: The man behind The Chipmunks was a
-------------------------
Q: Which Lloyd Webber musical premiered in the US on 10th December 1993? A:
True answ

Apart from all the answers being wrong, after the answer, the model continues to either repeat the question, ask new questions, and gives a separate answer for each newly posed question or provides more information about the answer. 

To account for this issue, the paper (page 16) proposes to trim all generations by pattern matching for the bad-words "Q:", "Question:", "QUESTION:", "questions:". This means that those tokens will not be part of the generation. Important to note is that the tokenization of for instance "Q:" and " Q:" is different (added space in front of Q), as the following example demonstrates. As a result, for every named bad-word, I also add " bad-word" as a bad-word. 

In [9]:
print(f"Tokenization for 'Q:': {tokenizer('Q:')}")
print(f"Tokenization for ' Q:': {tokenizer(' Q:')}")

example_answer = "The Chipmunks Q: Who was the man behind The Chipmunks?"
print(f"Excerpt of example answer{example_answer}")
tokenized_example_answer = tokenizer(example_answer)
print(f"Tokenized example answer: {tokenized_example_answer}")

Tokenization for 'Q:': {'input_ids': [2, 1864, 35], 'attention_mask': [1, 1, 1]}
Tokenization for ' Q:': {'input_ids': [2, 1209, 35], 'attention_mask': [1, 1, 1]}
Excerpt of example answerThe Chipmunks Q: Who was the man behind The Chipmunks?
Tokenized example answer: {'input_ids': [2, 133, 11055, 20614, 2258, 1209, 35, 3394, 21, 5, 313, 639, 20, 11055, 20614, 2258, 116], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


As you can see, the series 1209, 35 appears in the tokenized example answer, but the series 1864, 35 doesn't appear. 

As TriviaQA provides short answers in one line, the token of the new line character is specified as the eos token id. This means that the generation stops, once a \n is encountered. In addition, the paper (page 16) suggests to use few-shot prompting (n = 10) for TriviaQA. 

In [10]:
selected_training_data = data_trivia_train.select(range(0, 10))
ten_shot_prompt = ""

for data in selected_training_data:
    ten_shot_prompt += "Q: " + data["question"] + " A: " + data["answer"]["value"] + " "

# Define stop tokens, use token on position 1 bc position 0 is special token
stop_tokens = ["Q:", "Question:", "QUESTION:", "questions:", " Q:", " Question:", " QUESTION:", " questions:"]
stop_tokens = [[tokenizer(stop_token)["input_ids"][1]] for stop_token in stop_tokens]

# Define eos token
eos_token = tokenizer("\n")["input_ids"][1]

# Try again
for i in range(3):
    question = ten_shot_prompt + "Q: " + data_trivia_val[i]["question"] + " A:"
    answer = data_trivia_val[i]["answer"]["value"]

    inputs = tokenizer(question, padding=False, truncation=False, return_tensors="pt").to(device)
    length_input = inputs["input_ids"].shape[1]

    generate_ids = model.generate(inputs.input_ids,
                                  max_length=256 + length_input,
                                  eos_token_id=eos_token,
                                  bad_words_ids=stop_tokens)
    output = tokenizer.batch_decode(generate_ids[0][length_input:], skip_special_tokens=True)

    print(question)
    print(f"True answer: {answer}")
    print(f"Model Output: {''.join(output)}")

    print("-------------------------")

Q: Which American-born Sinclair won the Nobel Prize for Literature in 1930? A: Sinclair Lewis Q: Where in England was Dame Judi Dench born? A: York Q: In which decade did Billboard magazine first publish and American hit chart? A: 30s Q: From which country did Angola achieve independence in 1975? A: Portugal Q: Which city does David Soul come from? A: Chicago Q: Who won Super Bowl XX? A: Chicago Bears Q: Which was the first European country to abolish capital punishment? A: Norway Q: In which country did he widespread use of ISDN begin in 1988? A: Japan Q: What is Bruce Willis' real first name? A: Walter Q: Which William wrote the novel Lord Of The Flies? A: Golding Q: Who was the man behind The Chipmunks? A:
True answer: David Seville
Model Output:  Chipmunk creator Chipmunks creator Chipmunk creator Chipmunk creator Chipmunk creator Chipmunk creator Chipmunk creator Chipmunk creator Chipmunk creator Chipmunk creator Chipmunk creator Chipmunk creator Chipmunk creator Chipmunk creator 

As you can see, the answer is repeated in the first example. This could be solved by changing Q: and A: to Question: and Answer: and also adding "A:", " A:", "Answer:", "ANSWER:", "answers:" as stopwords. 

In [11]:
selected_training_data = data_trivia_train.select(range(0, 10))
# few shot prompting
ten_shot_prompt = ""

for data in selected_training_data:
    ten_shot_prompt += "Question: " + data["question"] + " Answer: " + data["answer"]["value"] + " "

# Define stop tokens, use token on position 1 bc position 0 is special token
stop_tokens = ["Q:", "Question:", "QUESTION:", "questions:", " Q:", " Question:", " QUESTION:", " questions:",
               "A:", "Answer:", "ANSWER:", "answers:", " A:", " Answer:", " ANSWER:", " answers:"]
stop_tokens = [[tokenizer(stop_token)["input_ids"][1]] for stop_token in stop_tokens]

# Define eos token
eos_token = tokenizer("\n")["input_ids"][1]

# Try again
for i in range(10):
    question = ten_shot_prompt + "Question: " + data_trivia_val[i]["question"] + " Answer:"
    answer = data_trivia_val[i]["answer"]["value"]

    inputs = tokenizer(question, padding=False, truncation=False, return_tensors="pt").to(device)
    length_input = inputs["input_ids"].shape[1]

    generate_ids = model.generate(inputs.input_ids,
                                  max_length=256 + length_input,
                                  eos_token_id=eos_token,
                                  bad_words_ids=stop_tokens)
    output = tokenizer.batch_decode(generate_ids[0][length_input:], skip_special_tokens=True)

    print(question)
    print(f"True answer: {answer}")
    print(f"Model Output: {''.join(output)}")

    print("-------------------------")

Question: Which American-born Sinclair won the Nobel Prize for Literature in 1930? Answer: Sinclair Lewis Question: Where in England was Dame Judi Dench born? Answer: York Question: In which decade did Billboard magazine first publish and American hit chart? Answer: 30s Question: From which country did Angola achieve independence in 1975? Answer: Portugal Question: Which city does David Soul come from? Answer: Chicago Question: Who won Super Bowl XX? Answer: Chicago Bears Question: Which was the first European country to abolish capital punishment? Answer: Norway Question: In which country did he widespread use of ISDN begin in 1988? Answer: Japan Question: What is Bruce Willis' real first name? Answer: Walter Question: Which William wrote the novel Lord Of The Flies? Answer: Golding Question: Who was the man behind The Chipmunks? Answer:
True answer: David Seville
Model Output:  Chip

-------------------------
Question: Which American-born Sinclair won the Nobel Prize for Literature i

The format of the answers looks good - we can argue about their correctness though (except for one example).

## Include probability of generated output sequence
As the last token is always eos token, I don't want to include this in my probability calculation. Therefore, the probabilities are only multiplied until the second to last token.

In [12]:
selected_training_data = data_trivia_train.select(range(0, 10))
# few shot prompting
ten_shot_prompt = ""

for data in selected_training_data:
    ten_shot_prompt += "Question: " + data["question"] + " Answer: " + data["answer"]["value"] + " "

# Define stop tokens, use token on position 1 bc position 0 is special token
stop_tokens = ["Q:", "Question:", "QUESTION:", "questions:", " Q:", " Question:", " QUESTION:", " questions:",
               "A:", "Answer:", "ANSWER:", "answers:", " A:", " Answer:", " ANSWER:", " answers:"]
stop_tokens = [[tokenizer(stop_token)["input_ids"][1]] for stop_token in stop_tokens]

# Define eos token
eos_token = tokenizer("\n")["input_ids"][1]

for i in range(10):
    question = ten_shot_prompt + "Question: " + data_trivia_val[i]["question"] + " Answer:"
    answer = data_trivia_val[i]["answer"]["value"]

    inputs = tokenizer(question, padding=False, truncation=False, return_tensors="pt").to(device)
    length_input = inputs["input_ids"].shape[1]

    output_generate = model.generate(inputs.input_ids,
                                     max_length=256 + length_input,
                                     eos_token_id=eos_token,
                                     bad_words_ids=stop_tokens,
                                     return_dict_in_generate=True,
                                     output_scores=True)
    # https://huggingface.co/docs/transformers/internal/generation_utils
    generate_ids = output_generate.sequences
    scores = output_generate.scores
    output = tokenizer.batch_decode(generate_ids[0][length_input:-1], skip_special_tokens=True)

    # Calculating probability of sequence
    prob_output = 1
    for j, token in enumerate(generate_ids[0][length_input:-1]):
        # Scores are prediction scores meaning scores for each vocabulary token before softmax
        probs_vocab = F.softmax(scores[j], dim=-1)
        prob_token = probs_vocab[0, token].item()
        prob_output *= prob_token

    print(question)
    print(f"True answer: {answer}")
    print(f"Model Output: {''.join(output)}")
    print(f"Probability of output: {prob_output}")

    print("-------------------------")

Question: Which American-born Sinclair won the Nobel Prize for Literature in 1930? Answer: Sinclair Lewis Question: Where in England was Dame Judi Dench born? Answer: York Question: In which decade did Billboard magazine first publish and American hit chart? Answer: 30s Question: From which country did Angola achieve independence in 1975? Answer: Portugal Question: Which city does David Soul come from? Answer: Chicago Question: Who won Super Bowl XX? Answer: Chicago Bears Question: Which was the first European country to abolish capital punishment? Answer: Norway Question: In which country did he widespread use of ISDN begin in 1988? Answer: Japan Question: What is Bruce Willis' real first name? Answer: Walter Question: Which William wrote the novel Lord Of The Flies? Answer: Golding Question: Who was the man behind The Chipmunks? Answer:
True answer: David Seville
Model Output:  Chip
Probability of output: 0.03923239931464195
-------------------------
Question: Which American-born Sin

## Sampling multiple answers
### Setup
The following setup remains the same for different sampling techniques. Sample 10 answers per question (see page 8).

See: https://huggingface.co/docs/transformers/en/generation_strategies

As answers like the one below appeared when trying to sample, the stop tokens "Answers:", " Answers:", "Topic:", " Topic:", "...", " ...", ".", " .", " ?", "?", ":", " :", "!", " !" are also added to the list. As the answers of TriviaQA are quite short and don't contain multiple sentences, the generated answers are more similar to the correct ones.

Sequence (0):  Chip Dursley Topic: Where/when is the most popular movie of all time filmed? Answers:I'm not sure...but I think you'll be good. I'd bring some books and keep an eye out for deals at Barnes & Noble, they have some cool gifts there. (P: 0.0)

Sequence (4):  Sherman Query: Which of the following US cities has a higher percentage of Hispanic population than any other in America? Chicago Texas Philadelphia Denver Los Angeles Miami Fort Worth Oklahoma City Atlanta Richmond Dallas Tampa (P: 0.0)

As the sampled tokenlist sometimes look like the following, when calculating the probability it is checked if the token is a newline character, an empty string or a special token like <pad> (identified by having a < and a >). If that's the case, it is not included in the probability calculation.

[' D', 'ery', 'ck', ' Gibson', '\n', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

In [7]:
# few shot prompting
selected_training_data = data_trivia_train.select(range(0, 10))
ten_shot_prompt = ""
for data in selected_training_data:
    ten_shot_prompt += "Question: " + data["question"] + " Answer: " + data["answer"]["value"] + " "

# Define stop tokens, use token on position 1 bc position 0 is special token
stop_tokens = ["Q:", "Question:", "QUESTION:", "questions:", " Q:", " Question:", " QUESTION:", " questions:",
               "A:", "Answer:", "ANSWER:", "answers:", " A:", " Answer:", " ANSWER:", " answers:", "Answers:", " Answers:", 
               "Topic:", " Topic:", ".", " .", "...", " ...", "?", " ?", ":", " :", "!", " !"]
stop_tokens = [[tokenizer(stop_token)["input_ids"][1]] for stop_token in stop_tokens]
stop_tokens.extend([tokenizer("\n")["input_ids"]])

# Define eos token
eos_token = tokenizer("\n")["input_ids"][1]

# How many questions to test
n_questions = 3

# How many answers to sample 
n_sample = 10

print(f"Few shot prompt:\n{ten_shot_prompt}")

Few shot prompt:
Question: Which American-born Sinclair won the Nobel Prize for Literature in 1930? Answer: Sinclair Lewis Question: Where in England was Dame Judi Dench born? Answer: York Question: In which decade did Billboard magazine first publish and American hit chart? Answer: 30s Question: From which country did Angola achieve independence in 1975? Answer: Portugal Question: Which city does David Soul come from? Answer: Chicago Question: Who won Super Bowl XX? Answer: Chicago Bears Question: Which was the first European country to abolish capital punishment? Answer: Norway Question: In which country did he widespread use of ISDN begin in 1988? Answer: Japan Question: What is Bruce Willis' real first name? Answer: Walter Question: Which William wrote the novel Lord Of The Flies? Answer: Golding 


In [8]:
def calculate_probability_sequence(sequence_ids, scores):
    prob_output = 1
    for j, token in enumerate(sequence_ids):
        # Don't include probability of new line, empty string, special token
        decoded_token = tokenizer.decode(token).strip()
        if decoded_token in ["", "\n", '', "<pad>"] or ("<" in decoded_token and ">" in decoded_token):
            continue
            
        # Scores are prediction scores meaning scores for each vocabulary token before softmax
        probs_vocab = F.softmax(scores[j], dim=-1)
        prob_token = probs_vocab[0, token].item()
        prob_output *= prob_token
    return prob_output

This prompt is added before all following questions. However, as this prompt is always the same and I want to keep things clear, only the question is printed out.

### Multinomial sampling

In [11]:
for i in range(n_questions):
    torch.cuda.empty_cache()
    question = ten_shot_prompt + "Question: " + data_trivia_val[i]["question"] + " Answer:"
    answer = data_trivia_val[i]["answer"]["value"]
    
    # As mentioned above, only print question
    print(f"Question: {data_trivia_val[i]['question']}")
    print(f"True Answer: {answer}")

    inputs = tokenizer(question, padding=False, truncation=False, return_tensors="pt").to(device)
    length_input = inputs["input_ids"].shape[1]
    
    # Most probable output
    output_generate = model.generate(inputs.input_ids,
                                     max_length=16 + length_input,
                                     eos_token_id=eos_token,
                                     bad_words_ids=stop_tokens,
                                     return_dict_in_generate=True,
                                     output_scores=True)
    # https://huggingface.co/docs/transformers/internal/generation_utils
    generate_ids = output_generate.sequences
    scores = output_generate.scores
    output = tokenizer.batch_decode(generate_ids[0][length_input:-1], skip_special_tokens=True)

    # Calculating probability of sequence
    prob_output = calculate_probability_sequence(generate_ids[0][length_input:-1], scores)
    
    print(f"Most probable answer: {''.join(output)} (P: {prob_output:.6})")
        
    # Sample other outputs
    print(f"Sampled answer sequences:")
    output_generate = model.generate(inputs.input_ids,
                                     max_length=16 + length_input,
                                     eos_token_id=eos_token,
                                     bad_words_ids=stop_tokens,
                                     return_dict_in_generate=True,
                                     output_scores=True,
                                     do_sample=True, 
                                     num_return_sequences = n_sample)
    
    # https://huggingface.co/docs/transformers/internal/generation_utils
    generate_ids = output_generate.sequences
    scores = output_generate.scores
    
    for n_sequence in range(n_sample):
        output = tokenizer.batch_decode(generate_ids[n_sequence][length_input:-1], skip_special_tokens=True)

        # Calculating probability of sequence
        prob_output = calculate_probability_sequence(generate_ids[n_sequence][length_input:-1], scores)

        # Print out result
        output_string = ''.join(output).replace("\n", "")
        print(f"Sequence ({n_sequence}): {output_string} (P: {prob_output:.6})")
    
    print("-------------------------")

Question: Who was the man behind The Chipmunks?
True Answer: David Seville
Most probable answer:  Chip (P: 0.0392474)
Sampled answer sequences:
Sequence (0):  Donner (P: 0.00555596)
Sequence (1):  Pillsbury CompanyThis guy is the poster child for someone who has (P: 0.0)
Sequence (2):  Warner BrothersI had the same question as you this morning before even getting (P: 0.0)
Sequence (3):  Micky mouse (P: 0.0)
Sequence (4):  Sandberg (P: 0.0)
Sequence (5):  The ChipmunksI still can't make my head around why any (P: 0.0)
Sequence (6):  Sandberg (P: 0.0)
Sequence (7):  The chipmunks Movie trivia answer is (P: 0.0)
Sequence (8):  Chris Rolfe (P: 0.0)
Sequence (9):  Daffy Duck (P: 0.0)
-------------------------
Question: Which Lloyd Webber musical premiered in the US on 10th December 1993?
True Answer: Sunset Boulevard
Most probable answer:  The Phantom Of The Opera (P: 0.0270808)
Sampled answer sequences:
Sequence (0):  London Palladium (or possibly West End) Playbill (P: 6.66469e-14)
Sequen

### Multinomial Beam Sampling

In [14]:
for i in range(n_questions):
    torch.cuda.empty_cache()
    question = ten_shot_prompt + "Question: " + data_trivia_val[i]["question"] + " Answer:"
    answer = data_trivia_val[i]["answer"]["value"]
    
    # As mentioned above, only print question
    print(f"Question: {data_trivia_val[i]['question']}")
    print(f"True Answer: {answer}")

    inputs = tokenizer(question, padding=False, truncation=False, return_tensors="pt").to(device)
    length_input = inputs["input_ids"].shape[1]
    
    # Most probable output
    output_generate = model.generate(inputs.input_ids,
                                     max_length=16 + length_input,
                                     eos_token_id=eos_token,
                                     bad_words_ids=stop_tokens,
                                     return_dict_in_generate=True,
                                     output_scores=True)
    # https://huggingface.co/docs/transformers/internal/generation_utils
    generate_ids = output_generate.sequences
    scores = output_generate.scores
    output = tokenizer.batch_decode(generate_ids[0][length_input:-1], skip_special_tokens=True)

    # Calculating probability of sequence
    prob_output = calculate_probability_sequence(generate_ids[0][length_input:-1], scores)
    
    print(f"Most probable answer: {''.join(output)} (P: {prob_output:.6})")
        
    # Sample other outputs
    print(f"Sampled answer sequences:")
    output_generate = model.generate(inputs.input_ids,
                                     max_length=16 + length_input,
                                     eos_token_id=eos_token,
                                     bad_words_ids=stop_tokens,
                                     return_dict_in_generate=True,
                                     output_scores=True,
                                     do_sample=True, 
                                     num_beams=2*n_sample,
                                     num_return_sequences = n_sample)
    
    # https://huggingface.co/docs/transformers/internal/generation_utils
    generate_ids = output_generate.sequences
    scores = output_generate.scores
    
    for n_sequence in range(n_sample):
        output = tokenizer.batch_decode(generate_ids[n_sequence][length_input:-1], skip_special_tokens=True)

        # Calculating probability of sequence
        prob_output = calculate_probability_sequence(generate_ids[n_sequence][length_input:-1], scores)

        # Print out result
        output_string = ''.join(output).replace("\n", "")
        print(f"Sequence ({n_sequence}): {output_string} (P: {prob_output:.6})")
    
    print("-------------------------")

Question: Who was the man behind The Chipmunks?
True Answer: David Seville
Most probable answer:  Chip (P: 0.0392474)
Sampled answer sequences:
Sequence (0):  The Chipmunks' creator, The Chipmunk's creator, The (P: 0.0)
Sequence (1):  The Chipmunks' creator, The Chipmunk's creator, Chip (P: 0.0)
Sequence (2):  The Chipmunks' creator, John Lasseter (P: 0.0)
Sequence (3):  The Chipmunks' creator, The Chipmunk's creator, and (P: 0.0)
Sequence (4):  The Chipmunks' creator Bill Plympton (P: 0.0)
Sequence (5):  The Chipmunks' creator, Theodor Geisel (P: 0.0)
Sequence (6):  The Chipmunks' creator, The Chipmunk's creator, and (P: 0.0)
Sequence (7):  The Chipmunks' creator, John LasseterI�� (P: 0.0)
Sequence (8):  The Chipmunks' creator, The Chipmunk's creator, Jim (P: 0.0)
Sequence (9):  The Chipmunks' creator, The Chipmunk's creator, Chuck (P: 0.0)
-------------------------
Question: Which Lloyd Webber musical premiered in the US on 10th December 1993?
True Answer: Sunset Boulevard
Most proba